# micro diffusion

https://huggingface.co/datasets/JourneyDB/JourneyDB

## 実装

### 環境構築

In [ ]:
import logging
import os
import platform
import subprocess

if os.path.exists("debug.log"):
    os.remove("debug.log")

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler("debug.log")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger.info(f"{NVIDIA_SMI}")

In [ ]:
%pip install \
    "mosaicml[tensorboard, wandb]" \
    accelerate \
    beautifulsoup4 \
    datasets \
    diffusers \
    easydict \
    einops \
    fastparquet \
    huggingface_hub \
    hydra-core \
    mosaicml-streaming \
    omegaconf \
    open_clip_torch \
    pandas \
    timm \
    torchmetrics \
    tqdm \
    transformers

In [ ]:
from accelerate import Accelerator
from collections.abc import Iterable
from composer import Trainer
from composer.algorithms import GradientClipping
from composer.algorithms.low_precision_layernorm import apply_low_precision_layernorm
from composer.callbacks import LRMonitor
from composer.callbacks import OptimizerMonitor
from composer.callbacks import RuntimeEstimator
from composer.callbacks import SpeedMonitor
from composer.core import Precision
from composer.loggers import TensorboardLogger
from composer.loggers.wandb_logger import WandBLogger
from composer.models import ComposerModel
from composer.optim import CosineAnnealingWithWarmupScheduler
from composer.optim import ConstantScheduler
from composer.utils import dist, reproducibility
from diffusers import AutoencoderKL
from diffusers import AutoencoderKL
from diffusers.models.modeling_outputs import AutoencoderKLOutput
from easydict import EasyDict
from einops import rearrange
from functools import partial
from glob import glob
from huggingface_hub import hf_hub_download
from hydra import compose, initialize_config_dir
from itertools import repeat
from micro_diffusion.models.callbacks import LogDiffusionImages
from micro_diffusion.models.callbacks import NaNCatcher
from micro_diffusion.models.utils import text_encoder_embedding_format
from multiprocessing import Pool
from omegaconf import DictConfig, OmegaConf
from PIL import Image, UnidentifiedImageError
from streaming import MDSWriter
from streaming import Stream, StreamingDataset
from streaming.base import MDSWriter
from streaming.base.util import merge_index
from timm.models.vision_transformer import PatchEmbed
from torch.utils.data import DataLoader
from torchmetrics import Metric
from torchvision import transforms
from tqdm import tqdm
from transformers import CLIPTextModel, CLIPTokenizer, T5EncoderModel, T5Tokenizer
from typing import Callable, Dict, List, Optional, Sequence, Union
from typing import List, Dict, Union, Optional
from typing import List, Optional
from typing import Optional, Tuple, Dict, Union, List, Any
import hydra
import json
import math
import numpy as np
import open_clip
import os
import shutil
import subprocess
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers

# cuDNN(CUDA Deep Neural Network library）による最適化を有効化
# 3-5%の速度向上が見込まれる
torch.backends.cudnn.benchmark = True

# 乱数シードの設定
seed = 18
reproducibility.seed_all(seed)

logger.info(f"PyTorch {torch.__version__}")
logger.info(f"Transformers {transformers.__version__}")

In [ ]:
USER_ROOT = os.path.expanduser("~")
CACHE_DIR = os.path.join(USER_ROOT, ".cache", "micro_diffusion")

DATA_DIR = os.path.join(CACHE_DIR, "data")
os.makedirs(DATA_DIR, exist_ok=True)
logger.info(f"{DATA_DIR=}")

MODEL_DIR = os.path.join(CACHE_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)
logger.info(f"{MODEL_DIR=}")

In [ ]:
DATA_TYPES = {
    'float16': torch.float16,
    'bfloat16': torch.bfloat16,
    'float32': torch.float32
}

## DiT

### ヘルパー関数

In [ ]:
def modulate(x: torch.Tensor, shift: torch.Tensor, scale: torch.Tensor) -> torch.Tensor:
    """Applies modulation to input tensor using shift and scale factors."""
    return x * (1 + scale.unsqueeze(1)) + shift.unsqueeze(1)

In [ ]:
def create_norm(norm_type: str, dim: int, eps: float = 1e-6) -> nn.Module:
    """Creates a normalization layer based on the specified type."""
    if norm_type == "layernorm":
        return nn.LayerNorm(dim, eps=eps, bias=False)
    elif norm_type == "np_layernorm":
        return nn.LayerNorm(dim, eps=eps, elementwise_affine=False, bias=False)
    else:
        raise ValueError('Norm type not supported!')

### SelfAttention

In [ ]:
class SelfAttention(nn.Module):
    """Self attention layer.
    
    Args:
        dim (int): Input and output tensor dimension
        num_heads (int): Number of attention heads
        qkv_bias (bool, True): Whether to use bias in QKV linear layers
        norm_eps (float, 1e-6): Epsilon for normalization layers
        hidden_dim (Optional[int], None): Dimension for qkv space. If None, same as input dim
    """
    def __init__(
        self,
        dim: int,
        num_heads: int,
        qkv_bias: bool = True,
        norm_eps: float = 1e-6,
        hidden_dim: Optional[int] = None
    ):
        super().__init__()
        self.dim = dim
        if hidden_dim is None:
            hidden_dim = dim
        assert hidden_dim % num_heads == 0, 'dim should be divisible by num_heads'
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.qkv_bias = qkv_bias

        self.qkv = nn.Linear(dim, hidden_dim * 3, bias=qkv_bias)
        self.proj = nn.Linear(hidden_dim, dim, bias=qkv_bias)

        self.ln_q = create_norm('np_layernorm', dim=hidden_dim, eps=norm_eps)
        self.ln_k = create_norm('np_layernorm', dim=hidden_dim, eps=norm_eps)

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.unbind(2)

        q = self.ln_q(q.view(B, N, self.num_heads * self.head_dim)).view(
            B, N, self.num_heads, self.head_dim).to(q.dtype)
        k = self.ln_k(k.view(B, N, self.num_heads * self.head_dim)).view(
            B, N, self.num_heads, self.head_dim).to(k.dtype)

        x = torch.nn.functional.scaled_dot_product_attention(
            q.transpose(1, 2),
            k.transpose(1, 2),
            v.transpose(1, 2),
            is_causal=False
        ).transpose(1, 2).contiguous()
        
        x = x.view(B, N, self.num_heads * self.head_dim)
        x = self.proj(x)
        return x
    
    def custom_init(self, init_std: float) -> None:
        nn.init.trunc_normal_(self.qkv.weight, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.proj.weight, mean=0.0, std=init_std)

### CrossAttention

In [ ]:
class CrossAttention(nn.Module):
    """Cross attention layer.
    
    Args:
        dim (int): Input and output tensor dimension
        num_heads (int): Number of attention heads
        qkv_bias (bool, True): Whether to use bias in QKV linear layers
        norm_eps (float, 1e-6): Epsilon for normalization layers
        hidden_dim (Optional[int], None): Dimension for qkv space. If None, same as input dim
    """
    def __init__(
        self,
        dim: int,
        num_heads: int,
        qkv_bias: bool = True,
        norm_eps: float = 1e-6,
        hidden_dim: Optional[int] = None
    ):
        super(CrossAttention, self).__init__()
        if hidden_dim is None:
            hidden_dim = dim
        assert hidden_dim % num_heads == 0, "dim must be divisible by num_heads"
        self.dim = dim
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.qkv_bias = qkv_bias

        self.q_linear = nn.Linear(dim, hidden_dim, bias=qkv_bias)
        self.kv_linear = nn.Linear(dim, hidden_dim*2, bias=qkv_bias)
        self.proj = nn.Linear(hidden_dim, dim, bias=qkv_bias)

        self.ln_q = create_norm('np_layernorm', dim=hidden_dim, eps=norm_eps)
        self.ln_k = create_norm('np_layernorm', dim=hidden_dim, eps=norm_eps)

    def forward(self, x: torch.Tensor, cond: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        B, N, C = x.shape
        q = self.q_linear(x).reshape(B, N, self.num_heads, self.head_dim)
        kv = self.kv_linear(cond).reshape(B, -1, 2, self.num_heads, self.head_dim)
        k, v = kv.unbind(2)

        q = self.ln_q(q.view(B, N, self.num_heads * self.head_dim)).view(
            B, N, self.num_heads, self.head_dim).to(q.dtype)
        k = self.ln_k(k.view(B, -1, self.num_heads * self.head_dim)).view(
            B, -1, self.num_heads, self.head_dim).to(k.dtype)

        x = torch.nn.functional.scaled_dot_product_attention(
            q.transpose(1, 2),
            k.transpose(1, 2),
            v.transpose(1, 2),
            is_causal=False
        ).transpose(1, 2).contiguous()
        
        x = x.view(B, -1, self.num_heads * self.head_dim)
        x = self.proj(x)
        return x
    
    def custom_init(self, init_std: float) -> None:
        for linear in (self.q_linear, self.kv_linear):
            nn.init.trunc_normal_(linear.weight, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.proj.weight, mean=0.0, std=init_std)

### MLP

In [ ]:
# Ref: https://github.com/huggingface/pytorch-image-models/blob/main/timm/layers/mlp.py
class Mlp(nn.Module):
    """MLP implementation from timm (without the dropout layers)
    
    Args:
        in_features (int): Input tensor dimension
        hidden_features (Optional[int], None): Number of hidden features. If None, same as in_features
        out_features (Optional[int], None): Number of output features. If None, same as in_features
        act_layer (Any, nn.GELU): Activation layer constructor. Defaults to GELU with tanh approximation
        norm_layer (Optional[Any], None): Normalization layer constructor. If None, uses Identity
        bias (bool, True): Whether to use bias in linear layers
    """
    def __init__(
        self,
        in_features: int,
        hidden_features: Optional[int] = None,
        out_features: Optional[int] = None,
        act_layer: Any = lambda: nn.GELU(approximate="tanh"),
        norm_layer: Optional[Any] = None,
        bias: bool = True,
    ):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features

        self.fc1 = nn.Linear(in_features, hidden_features, bias=bias)
        self.act = act_layer()
        self.norm = norm_layer if norm_layer is not None else nn.Identity()
        self.fc2 = nn.Linear(hidden_features, out_features, bias=bias)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.act(x)
        x = self.norm(x)
        x = self.fc2(x)
        return x

In [ ]:
class FeedForward(nn.Module):
    """Feed-forward block with SiLU activation.
    
    Args:
        dim (int): Input and output dimension
        hidden_dim (int): Hidden dimension betwen the two linear layers
        multiple_of (int): Round hidden dimension up to nearest multiple of this value
        use_bias (bool): Whether to use bias terms in linear layers
    """
    def __init__(
        self,
        dim: int,
        hidden_dim: int,
        multiple_of: int,
        use_bias: bool,
    ):
        super().__init__()
        self.dim = dim
        hidden_dim = int(2 * hidden_dim / 3)
        self.hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)
        
        self.w1 = nn.Linear(dim, self.hidden_dim, bias=use_bias)
        self.w2 = nn.Linear(dim, self.hidden_dim, bias=use_bias)
        self.w3 = nn.Linear(self.hidden_dim, dim, bias=use_bias)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.w3(F.silu(self.w1(x)) * self.w2(x))

    def custom_init(self, init_std: float) -> None:
        nn.init.trunc_normal_(self.w1.weight, mean=0.0, std=0.02)
        for linear in (self.w2, self.w3):
            nn.init.trunc_normal_(linear.weight, mean=0.0, std=init_std)

In [ ]:
class FeedForwardECMoe(nn.Module):
    """Expert-Choice style Mixture of Experts feed-forward layer with GELU activation.
    
    Args:
        num_experts (int): Number of experts in the layer
        expert_capacity (float): Capacity factor determining tokens per expert
        dim (int): Input and output dimension
        hidden_dim (int): Hidden dimension between the two linear layers
        multiple_of (int): Round hidden dimension up to nearest multiple of this value
    """
    def __init__(
        self,
        num_experts: int,
        expert_capacity: float,
        dim: int,
        hidden_dim: int,
        multiple_of: int,
    ):
        super().__init__()
        self.num_experts = num_experts
        self.expert_capacity = expert_capacity
        self.dim = dim
        self.hidden_dim = multiple_of * ((hidden_dim + multiple_of - 1) // multiple_of)

        self.w1 = nn.Parameter(torch.ones(num_experts, dim, self.hidden_dim))
        self.w2 = nn.Parameter(torch.ones(num_experts, self.hidden_dim, dim))
        self.gate = nn.Linear(dim, num_experts, bias=False)
        self.gelu = nn.GELU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        assert len(x.shape) == 3
        n, t, d = x.shape
        tokens_per_expert = int(self.expert_capacity * t / self.num_experts)

        scores = self.gate(x)  # [n, t, e]
        probs = F.softmax(scores, dim=-1)  # [n, t, e]
        g, m = torch.topk(probs.permute(0, 2, 1), tokens_per_expert, dim=-1)  # [n, e, k], [n, e, k]
        p = F.one_hot(m, num_classes=t).float()  # [n, e, k, t]

        xin = torch.einsum('nekt, ntd -> nekd', p, x)  # [n, e, k, d]
        h = torch.einsum('nekd, edf -> nekf', xin, self.w1)  # [n, e, k, 4d]
        h = self.gelu(h)
        h = torch.einsum('nekf, efd -> nekd', h, self.w2)  # [n, e, k, d]

        out = g.unsqueeze(dim=-1) * h  # [n, e, k, d]
        out = torch.einsum('nekt, nekd -> ntd', p, out)
        return out
    
    def custom_init(self, init_std: float):
        nn.init.trunc_normal_(self.gate.weight, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.w1, mean=0.0, std=0.02)
        nn.init.trunc_normal_(self.w2, mean=0.0, std=init_std)


### TImestepEmbedder

In [ ]:
class TimestepEmbedder(nn.Module):
    """Embeds scalar timesteps into vector representations.
    
    Args:
        hidden_size (int): Size of hidden dimension
        act_layer (Any): Activation layer constructor
        frequency_embedding_size (int, 512): Size of frequency embedding
    """
    def __init__(
        self,
        hidden_size: int,
        act_layer: Any,
        frequency_embedding_size: int = 512
    ):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(frequency_embedding_size, hidden_size, bias=True),
            act_layer(),
            nn.Linear(hidden_size, hidden_size, bias=True),
        )
        self.frequency_embedding_size = frequency_embedding_size

    @staticmethod
    def timestep_embedding(t: torch.Tensor, dim: int, max_period: int = 10000) -> torch.Tensor:
        """Create sinusoidal timestep embeddings."""
        half = dim // 2
        freqs = torch.exp(
            -math.log(max_period) * torch.arange(
                start=0,
                end=half,
                dtype=torch.float32,
                device=t.device
            ) / half
        )
        args = t[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat([embedding, torch.zeros_like(embedding[:, :1])], dim=-1)
        return embedding

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        t_freq = self.timestep_embedding(t, self.frequency_embedding_size).to(self.dtype)
        return self.mlp(t_freq)

    @property
    def dtype(self) -> torch.dtype:
        return next(self.parameters()).dtype

### CaptionProjection

In [ ]:
class CaptionProjection(nn.Module):
    """Projects caption embeddings to model dimension.
    
    Args:
        in_channels (int): Number of input channels
        hidden_size (int): Size of hidden dimension
        act_layer (Any): Activation layer constructor
        norm_layer (Any): Normalization layer constructor
    """
    def __init__(
        self,
        in_channels: int,
        hidden_size: int,
        act_layer: Any,
        norm_layer: Any
    ) -> None:
        super().__init__()
        self.y_proj = Mlp(
            in_features=in_channels,
            hidden_features=hidden_size,
            out_features=hidden_size,
            act_layer=act_layer,
            norm_layer=norm_layer
        )
    
    def forward(self, caption: torch.Tensor) -> torch.Tensor:
        return self.y_proj(caption)

### AttentionBlockPromptEmbedding

In [ ]:
class AttentionBlockPromptEmbedding(nn.Module):
    """Attention block specifically for processing prompt embeddings.
    
    Args:
        dim (int): Input and output dimension
        head_dim (int): Dimension size per attention head
        mlp_ratio (float): Multiplier for feed-forward network hidden dimension w.r.t input dim
        multiple_of (int): Round feed-forward network hidden dimension up to nearest multiple of this value
        norm_eps (float): Epsilon value for layer normalization
        use_bias (bool): Whether to use bias terms in linear layers
    """
    def __init__(
        self,
        dim: int,
        head_dim: int, 
        mlp_ratio: float,
        multiple_of: int,
        norm_eps: float,
        use_bias: bool,
    ) -> None:
        super().__init__()
        assert dim % head_dim == 0, 'Hidden dimension must be divisible by head dim'
        
        self.dim = dim
        self.num_heads = dim // head_dim
        
        self.norm1 = create_norm('layernorm', dim, eps=norm_eps)
        self.attn = SelfAttention(
            dim=dim,
            num_heads=self.num_heads,
            qkv_bias=use_bias,
            norm_eps=norm_eps,
        )
        self.norm2 = create_norm('layernorm', dim, eps=norm_eps)
        self.mlp = FeedForward(
            dim=dim,
            hidden_dim=int(dim * mlp_ratio),
            multiple_of=multiple_of,
            use_bias=use_bias,
        )

    def forward(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

    def custom_init(self, init_std: float = 0.02) -> None:
        self.attn.custom_init(init_std)
        self.mlp.custom_init(init_std)

### DiTBlock

In [ ]:
class DiTBlock(nn.Module):
    """DiT transformer block comprising Attention and MLP blocks. It supports choosing between 
     dense feed-forward and expert-choice style Mixture-of-Experts feed-forward blocks.
    
    Args:
        dim (int): Input and output dimension of the block
        head_dim (int): Dimension of each attention head
        mlp_ratio (float): Ratio for hidden dimension between linear layers in MLP block
        qkv_ratio (float): Ratio for dimension in qkv layers in attention block
        multiple_of (int): Round hidden dimensions up to nearest multiple of this value in MLP block
        pooled_emb_dim (int): Dimension of pooled caption embeddings
        norm_eps (float): Epsilon for layer normalization
        depth_init (bool): Whether to initialize weights of the last layer in MLP/Attention block based on block index
        layer_id (int): Index of this block in the dit model
        num_layers (int): Total number of blocks in the dit model
        compress_xattn (bool): Whether to scale cross-attention qkv dimension using qkv_ratio 
        use_bias (bool): Whether to use bias in linear layers
        moe_block (bool): Whether to use mixture of experts for MLP block
        num_experts (int): Number of experts if using MoE block
        expert_capacity (float): Capacity factor for each expert if using MoE block
    """
    def __init__(
        self,
        dim: int,
        head_dim: int,
        mlp_ratio: float,
        qkv_ratio: float,
        multiple_of: int,
        pooled_emb_dim: int,
        norm_eps: float,
        depth_init: bool,
        layer_id: int,
        num_layers: int,
        compress_xattn: bool,
        use_bias: bool,
        moe_block: bool,
        num_experts: int,
        expert_capacity: float,
    ):
        super().__init__()
        self.dim = dim
        qkv_hidden_dim = (
            (head_dim * 2) * ((int(dim * qkv_ratio) + head_dim * 2 - 1) // (head_dim * 2))
            if qkv_ratio != 1 else dim
        )
        mlp_hidden_dim = int(dim * mlp_ratio)

        self.norm1 = create_norm('layernorm', dim, eps=norm_eps)
        self.attn = SelfAttention(
            dim=dim,
            num_heads=qkv_hidden_dim // head_dim,
            qkv_bias=use_bias,
            norm_eps=norm_eps,
            hidden_dim=qkv_hidden_dim,
        )
        self.cross_attn = CrossAttention(
            dim=dim,
            num_heads=qkv_hidden_dim // head_dim if compress_xattn else dim // head_dim,
            qkv_bias=use_bias,
            norm_eps=norm_eps,
            hidden_dim=qkv_hidden_dim if compress_xattn else dim,
        )
        self.norm2 = create_norm('layernorm', dim, eps=norm_eps)
        self.norm3 = create_norm('layernorm', dim, eps=norm_eps)
        
        self.mlp = (
            FeedForwardECMoe(num_experts, expert_capacity, dim, mlp_hidden_dim, multiple_of)
            if moe_block else
            FeedForward(dim, mlp_hidden_dim, multiple_of, use_bias)
        )

        self.adaLN_modulation = nn.Sequential(
            nn.GELU(approximate="tanh"),
            nn.Linear(pooled_emb_dim, 6 * dim, bias=True),
        )
        
        self.weight_init_std = (
            0.02 / (2 * (layer_id + 1)) ** 0.5 if depth_init else
            0.02 / (2 * num_layers) ** 0.5
        )
        
    def forward(self, x: torch.Tensor, y: torch.Tensor, c: torch.Tensor, **kwargs) -> torch.Tensor:
        shift_msa, scale_msa, gate_msa, shift_mlp, scale_mlp, gate_mlp = (
            self.adaLN_modulation(c).chunk(6, dim=1)
        )
        x = x + gate_msa.unsqueeze(1) * self.attn(modulate(self.norm1(x), shift_msa, scale_msa))
        x = x + self.cross_attn(self.norm2(x), y)
        x = x + gate_mlp.unsqueeze(1) * self.mlp(modulate(self.norm3(x), shift_mlp, scale_mlp))
        return x

    def custom_init(self):
        for norm in (self.norm1, self.norm2, self.norm3):
            norm.reset_parameters()
        self.attn.custom_init(self.weight_init_std)
        self.cross_attn.custom_init(self.weight_init_std)
        self.mlp.custom_init(self.weight_init_std)

### T2IFinalLayer

In [ ]:
class T2IFinalLayer(nn.Module):
    """The final layer of DiT architecture.
    
    Args:
        hidden_size (int): Size of hidden dimension
        time_emb_dim (int): Dimension of timestep embeddings
        patch_size (int): Size of image patches 
        out_channels (int): Number of output channels
        act_layer (Any): Activation layer constructor
        norm_final (nn.Module): Final normalization layer
    """
    def __init__(
        self,
        hidden_size: int,
        time_emb_dim: int,
        patch_size: int,
        out_channels: int,
        act_layer: Any,
        norm_final: nn.Module
    ):
        super().__init__()
        self.linear = nn.Linear(
            hidden_size,
            patch_size * patch_size * out_channels,
            bias=True
        )
        self.adaLN_modulation = nn.Sequential(
            act_layer(),
            nn.Linear(time_emb_dim, 2 * hidden_size, bias=True)
        )
        self.norm_final = norm_final

    def forward(self, x: torch.Tensor, c: torch.Tensor) -> torch.Tensor:
        shift, scale = self.adaLN_modulation(c).chunk(2, dim=1)
        x = modulate(self.norm_final(x), shift, scale)
        x = self.linear(x)
        return x

### DiT

In [ ]:
def get_1d_sincos_pos_embed_from_grid(embed_dim: int, pos: np.ndarray) -> np.ndarray:
    """Get 1D sinusoidal positional embeddings from grid."""
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float64)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out)  # (M, D/2)
    emb_cos = np.cos(out)  # (M, D/2)

    return np.concatenate([emb_sin, emb_cos], axis=1)

In [ ]:
def get_2d_sincos_pos_embed_from_grid(embed_dim: int, grid: np.ndarray) -> np.ndarray:
    """Get 2D sinusoidal positional embeddings from grid."""
    assert embed_dim % 2 == 0

    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])

    return np.concatenate([emb_h, emb_w], axis=1)

In [ ]:
def ntuple(n: int):
    """Converts input into n-tuple."""
    def parse(x):
        if isinstance(x, Iterable) and not isinstance(x, str):
            return x
        return tuple(repeat(x, n))
    return parse

In [ ]:
def get_2d_sincos_pos_embed(
    embed_dim: int,
    grid_size: Union[int, Tuple[int, int]],
    cls_token: bool = False,
    extra_tokens: int = 0,
    pos_interp_scale: float = 1.0,
    base_size: int = 16
) -> np.ndarray:
    """Get 2D sinusoidal positional embeddings."""
    to_2tuple = ntuple(2)
    if isinstance(grid_size, int):
        grid_size = to_2tuple(grid_size)
    # Interpolate position embeddings to adapt model across resolutions. Interestingly, without any interpolation
    # the model does converge slowly at start (~1000 steps) but eventually achieves near similar qualitative performance.
    grid_h = np.arange(grid_size[0], dtype=np.float32) / (grid_size[0] / base_size) / pos_interp_scale
    grid_w = np.arange(grid_size[1], dtype=np.float32) / (grid_size[1] / base_size) / pos_interp_scale
    grid = np.meshgrid(grid_w, grid_h)
    grid = np.stack(grid, axis=0)
    grid = grid.reshape([2, 1, grid_size[1], grid_size[0]])

    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token and extra_tokens > 0:
        pos_embed = np.concatenate([np.zeros([extra_tokens, embed_dim]), pos_embed], axis=0)
    return pos_embed

In [ ]:
def mask_out_token(x: torch.Tensor, ids_keep: torch.Tensor) -> torch.Tensor:
    """Mask out tokens specified by ids_keep."""
    N, L, D = x.shape  # batch, length, dim
    x_masked = torch.gather(
        x,
        dim=1,
        index=ids_keep.unsqueeze(-1).repeat(1, 1, D)
    )
    return x_masked

In [ ]:
def get_mask(batch: int, length: int, mask_ratio: float, device: torch.device) -> Dict[str, torch.Tensor]:
    """Get binary mask for input sequence. 
    
    mask: binary mask, 0 is keep, 1 is remove
    ids_keep: indices of tokens to keep
    ids_restore: indices to restore the original order
    """
    len_keep = int(length * (1 - mask_ratio))
    noise = torch.rand(batch, length, device=device)  # noise in [0, 1]
    ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
    ids_restore = torch.argsort(ids_shuffle, dim=1)
    # keep the first subset
    ids_keep = ids_shuffle[:, :len_keep]

    mask = torch.ones([batch, length], device=device)
    mask[:, :len_keep] = 0
    mask = torch.gather(mask, dim=1, index=ids_restore)
    return {
        'mask': mask,
        'ids_keep': ids_keep,
        'ids_restore': ids_restore
    }

In [ ]:
def unmask_tokens(x: torch.Tensor, ids_restore: torch.Tensor, mask_token: torch.Tensor) -> torch.Tensor:
    """Unmask tokens using provided mask token."""
    mask_tokens = mask_token.repeat(x.shape[0], ids_restore.shape[1] - x.shape[1], 1)
    x_ = torch.cat([x, mask_tokens], dim=1)
    x_ = torch.gather(
        x_,
        dim=1,
        index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2])
    )  # unshuffle
    return x_

In [ ]:
class DiT(nn.Module):
    """
    Diffusion Transformer (DiT) model than support conditioning on caption embeddings for text-to-image generation.
    
    Args:
        input_size (int, default: 32): Size of input image (assumed square)
        patch_size (int, default: 2): Size of patches for patch embedding
        in_channels (int, default: 4): Number of input image channels (by default assuming four channel latent space)
        dim (int, default: 1152): Dimension of transformer backbone, i.e., dimension of major transformer layers
        depth (int, default: 28): Number of transformer blocks
        head_dim (int, default: 64): Dimension of each attention head
        multiple_of (int, default: 256): Round hidden dimensions up to nearest multiple of this value in MLP block
        caption_channels (int, default: 4096): Number of channels in caption embeddings
        pos_interp_scale (float, default: 1.0): Scale for positional embedding interpolation (1.0 for 256x256, 2.0 for 512x512)
        norm_eps (float, default: 1e-6): Epsilon for layer normalization
        depth_init (bool, default: True): Whether to use depth-dependent initialization in DiT blocks
        qkv_multipliers (List[float], default: [1.0]): Multipliers for QKV projection dimensions in DiT blocks
        ffn_multipliers (List[float], default: [4.0]): Multipliers for FFN hidden dimensions in DiT blocks
        use_patch_mixer (bool, default: True): Whether to use patch mixer layers
        patch_mixer_depth (int, default: 4): Number of patch mixer blocks
        patch_mixer_dim (int, default: 512): Dimension of patch-mixer layers
        patch_mixer_qkv_ratio (float, default: 1.0): Multipliers for QKV projection dimensions in patch-mixer blocks
        patch_mixer_mlp_ratio (float, default: 1.0): Multipliers for FFN hidden dimensions in patch-mixer blocks
        use_bias (bool, default: True): Whether to use bias in linear layers
        num_experts (int, default: 8):  Number of experts if using MoE block
        expert_capacity (int, default: 1): Capacity factor for each expert if using MoE FFN layers
        experts_every_n (int, default: 2): Add MoE FFN layers every n blocks
    """
    def __init__(
        self,
        input_size: int = 32,
        patch_size: int = 2,
        in_channels: int = 4,
        dim: int = 1152,
        depth: int = 28,
        head_dim: int = 64,
        multiple_of: int = 256,
        caption_channels: int = 1024,
        pos_interp_scale: float = 1.0,
        norm_eps: float = 1e-6,
        depth_init: bool = True,
        qkv_multipliers: List[float] = [1.0],
        ffn_multipliers: List[float] = [4.0],
        use_patch_mixer: bool = True,
        patch_mixer_depth: int = 4,
        patch_mixer_dim: int = 512,
        patch_mixer_qkv_ratio: float = 1.0,
        patch_mixer_mlp_ratio: float = 1.0,
        use_bias: bool = True,
        num_experts: int = 8,
        expert_capacity: int = 1,
        experts_every_n: int = 2
    ):
        super().__init__()
        self.input_size = input_size
        self.in_channels = in_channels
        self.out_channels = in_channels
        self.patch_size = patch_size
        self.head_dim = head_dim
        self.pos_interp_scale = pos_interp_scale
        self.use_patch_mixer = use_patch_mixer
        
        approx_gelu = lambda: nn.GELU(approximate="tanh")
        self.x_embedder = PatchEmbed(
            input_size, patch_size, in_channels, dim, bias=True
        )
        self.t_embedder = TimestepEmbedder(dim, approx_gelu)

        num_patches = self.x_embedder.num_patches
        self.base_size = input_size // self.patch_size
        self.register_buffer("pos_embed", torch.zeros(1, num_patches, dim))

        self.y_embedder = CaptionProjection(
            in_channels=caption_channels,
            hidden_size=dim,
            act_layer=approx_gelu,
            norm_layer=create_norm('layernorm', dim, eps=norm_eps)
        )

        self.y_emb_preprocess = AttentionBlockPromptEmbedding(
            dim,
            head_dim,
            mlp_ratio=4.0,
            multiple_of=multiple_of,
            norm_eps=norm_eps,
            use_bias=use_bias
        )
        
        self.pooled_y_emb_process = Mlp(
            dim,
            dim,
            dim,
            approx_gelu,
            norm_layer=create_norm('layernorm', dim, eps=norm_eps)
        )

        if self.use_patch_mixer:
            expert_blocks_idx = [
                i for i in range(1, patch_mixer_depth) 
                if (i+1) % experts_every_n == 0
            ]
            is_moe_block = [
                True if i in expert_blocks_idx else False 
                for i in range(patch_mixer_depth)
            ]
                 
            self.patch_mixer = nn.ModuleList([
                DiTBlock(
                    dim=patch_mixer_dim,
                    head_dim=head_dim,
                    mlp_ratio=patch_mixer_mlp_ratio,
                    qkv_ratio=patch_mixer_qkv_ratio,
                    multiple_of=multiple_of,
                    pooled_emb_dim=dim,
                    norm_eps=norm_eps,
                    depth_init=False,
                    layer_id=0,
                    num_layers=depth,
                    compress_xattn=False,
                    use_bias=use_bias,
                    moe_block=is_moe_block[i],
                    num_experts=num_experts,
                    expert_capacity=expert_capacity
                ) for i in range(patch_mixer_depth)
            ])

            # Couple of projection layers
            if patch_mixer_dim != dim:
                self.patch_mixer_map_xin = nn.Sequential(
                    create_norm('layernorm', dim, eps=norm_eps),
                    nn.Linear(dim, patch_mixer_dim, bias=use_bias)
                )
                self.patch_mixer_map_xout = nn.Sequential(
                    create_norm('layernorm', patch_mixer_dim, eps=norm_eps),
                    nn.Linear(patch_mixer_dim, dim, bias=use_bias)
                )
                self.patch_mixer_map_y = nn.Sequential(
                    create_norm('layernorm', dim, eps=norm_eps),
                    nn.Linear(dim, patch_mixer_dim, bias=use_bias)
                )
            else:
                self.patch_mixer_map_xin = nn.Identity()
                self.patch_mixer_map_xout = nn.Identity()
                self.patch_mixer_map_y = nn.Identity()

        assert len(ffn_multipliers) == len(qkv_multipliers)
        if len(ffn_multipliers) == depth:
            qkv_ratios = qkv_multipliers
            mlp_ratios = ffn_multipliers
        else:
            # Distribute the multiplers across each partition
            num_splits = len(ffn_multipliers)
            assert depth % num_splits == 0, 'number of blocks should be divisible by number of splits'
            depth_per_split = depth // num_splits
            qkv_ratios = list(np.array([
                [m]*depth_per_split for m in qkv_multipliers
            ]).reshape(-1))
            mlp_ratios = list(np.array([
                [m]*depth_per_split for m in ffn_multipliers
            ]).reshape(-1))

        # Don't use MoE in last block
        expert_blocks_idx = [
            i for i in range(0, depth - 1) 
            if (i+1) % experts_every_n == 0
        ]
        is_moe_block = [
            True if i in expert_blocks_idx else False 
            for i in range(depth)
        ]
        
        self.blocks = nn.ModuleList([
            DiTBlock(
                dim=dim,
                head_dim=head_dim,
                mlp_ratio=mlp_ratios[i],
                qkv_ratio=qkv_ratios[i],
                multiple_of=multiple_of,
                pooled_emb_dim=dim,
                norm_eps=norm_eps,
                depth_init=depth_init,
                layer_id=i,
                num_layers=depth,
                compress_xattn=False,
                use_bias=use_bias,
                moe_block=is_moe_block[i],
                num_experts=num_experts,
                expert_capacity=expert_capacity
            ) for i in range(depth)
        ])
        
        self.register_buffer(
            "mask_token", 
            torch.zeros(1, 1, patch_size ** 2 * self.out_channels)
        )
        self.final_layer = T2IFinalLayer(
            dim,
            dim,
            patch_size,
            self.out_channels,
            approx_gelu,
            create_norm('layernorm', dim, eps=norm_eps)
        )

        self.initialize_weights()

    def forward_without_cfg(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
        y: torch.Tensor,
        mask_ratio: float = 0,
        **kwargs
    ) -> dict:
        """Forward pass without classifier-free guidance.

        Args:
            x: Input tensor of shape (batch_size, channels, height, width)
            t: Timestep tensor of shape (batch_size,)
            y: Caption embedding tensor of shape (batch_size, 1, seq_len, dim)
            mask_ratio: Ratio of patches to mask during training (between 0 and 1)

        Returns:
            dict: Dictionary containing:
                - 'sample': Output tensor of shape (batch_size, out_channels, height, width)
                - 'mask': Optional binary mask tensor, if masking was applied else None
        """
        self.h = x.shape[-2] // self.patch_size
        self.w = x.shape[-1] // self.patch_size

        x = self.x_embedder(x) + self.pos_embed  # (N, T, D), where T = H * W / patch_size ** 2
        t = self.t_embedder(t.expand(x.shape[0]))  # (N, D)

        y = self.y_embedder(y)  # (N, 1, L, D)
        y = self.y_emb_preprocess(y.squeeze(dim=1)).unsqueeze(dim=1)  # (N, 1, L, D) -> (N, D)
        y_pooled = self.pooled_y_emb_process(y.mean(dim=-2).squeeze(dim=1))
        t = t + y_pooled

        mask = None
        
        if self.use_patch_mixer:
            x = self.patch_mixer_map_xin(x)
            y_mixer = self.patch_mixer_map_y(y)
            for block in self.patch_mixer:
                x = block(x, y_mixer, t)  # (N, T, D_mixer)
        
        if mask_ratio > 0:
            mask_dict = get_mask(
                x.shape[0], x.shape[1], 
                mask_ratio=mask_ratio, 
                device=x.device
            )
            ids_keep = mask_dict['ids_keep']
            ids_restore = mask_dict['ids_restore']
            mask = mask_dict['mask']
            x = mask_out_token(x, ids_keep)
        
        if self.use_patch_mixer:
            # Project mixer out to backbone transformer dim (do after masking to save compute)
            x = self.patch_mixer_map_xout(x)

        for block in self.blocks:
            x = block(x, y, t)  # (N, T, D)
        
        x = self.final_layer(x, t)  # (N, T, patch_size ** 2 * out_channels)
        
        if mask_ratio > 0:
            x = unmask_tokens(x, ids_restore, self.mask_token)

        x = self.unpatchify(x)  # (N, out_channels, H, W)
        return {'sample': x, 'mask': mask}
    
    def forward_with_cfg(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
        y: torch.Tensor,
        cfg: float = 1.0,
        mask_ratio: float = 0,
        **kwargs
    ) -> dict:
        """Forward pass with classifier-free guidance.
        
        Args:
            x: Input tensor of shape (batch_size, channels, height, width)
            t: Timestep tensor of shape (batch_size,)
            y: Caption embedding tensor of shape (batch_size, 1, seq_len, dim)
            cfg: Classifier-free guidance scale (1.0 means no guidance)
            mask_ratio: Ratio of patches to mask during training (between 0 and 1)

        Returns:
            dict: Dict with output tensor of shape (batch_size, out_channels, height, width)
        """
        x = torch.cat([x, x], 0)
        y = torch.cat([y, torch.zeros_like(y)], 0)
        if len(t) != 1:
            t = torch.cat([t, t], 0)
        
        eps = self.forward_without_cfg(x, t, y, mask_ratio, **kwargs)['sample']
        cond_eps, uncond_eps = torch.split(eps, len(eps) // 2, dim=0)
        eps = uncond_eps + cfg * (cond_eps - uncond_eps)
        return {'sample': eps}

    def forward(
        self,
        x: torch.Tensor,
        t: torch.Tensor,
        y: torch.Tensor,
        cfg: float = 1.0,
        **kwargs
    ) -> dict:
        """Routes to appropriate forward pass based on classifier-free guidance value."""
        if cfg != 1.0:
            return self.forward_with_cfg(x, t, y, cfg, **kwargs)
        else:
            return self.forward_without_cfg(x, t, y, **kwargs)

    def unpatchify(self, x: torch.Tensor) -> torch.Tensor:
        """Reverses the patch embedding process to reconstruct the original image dimensions."""
        c = self.out_channels
        p = self.x_embedder.patch_size[0]
        h = w = int(x.shape[1]**0.5)
        assert h * w == x.shape[1]

        x = x.reshape(shape=(x.shape[0], h, w, p, p, c))
        x = torch.einsum('nhwpqc->nchpwq', x)
        return x.reshape(shape=(x.shape[0], c, h * p, h * p))

    def initialize_weights(self) -> None:
        """Initialize model weights with custom initialization scheme."""
        def zero_bias(m: nn.Module) -> None:
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)

        def _basic_init(module: nn.Module) -> None:
            if isinstance(module, nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                zero_bias(module)  # All bias in the model init to zero

        # Baseline init of all parameters
        self.apply(_basic_init)

        pos_embed = get_2d_sincos_pos_embed(
            self.pos_embed.shape[-1],
            int(self.x_embedder.num_patches**0.5),
            pos_interp_scale=self.pos_interp_scale,
            base_size=self.base_size
        )
        self.pos_embed.data.copy_(torch.from_numpy(pos_embed).float().unsqueeze(0))

        w = self.x_embedder.proj.weight.data
        nn.init.xavier_uniform_(w.view([w.shape[0], -1]))

        nn.init.normal_(self.t_embedder.mlp[0].weight, std=0.02)
        nn.init.normal_(self.t_embedder.mlp[2].weight, std=0.02)
        nn.init.normal_(self.pooled_y_emb_process.fc1.weight, std=0.02)
        nn.init.normal_(self.pooled_y_emb_process.fc2.weight, std=0.02)
        nn.init.normal_(self.y_embedder.y_proj.fc1.weight, std=0.02)
        nn.init.normal_(self.y_embedder.y_proj.fc2.weight, std=0.02)
        
        # Custom init of blocks
        for block in self.blocks:
            block.custom_init()
        for block in self.patch_mixer:
            block.custom_init()

        for block in self.blocks:
            nn.init.constant_(block.adaLN_modulation[-1].weight, 0)

        for block in self.patch_mixer:
            nn.init.constant_(block.adaLN_modulation[-1].weight, 0)

        self.y_emb_preprocess.custom_init()
        nn.init.constant_(self.y_emb_preprocess.attn.proj.weight, 0)
        nn.init.constant_(self.y_emb_preprocess.mlp.w3.weight, 0)

        # Zero-out output layers
        nn.init.constant_(self.final_layer.adaLN_modulation[-1].weight, 0)
        nn.init.constant_(self.final_layer.linear.weight, 0)


In [ ]:
def MicroDiT_Tiny_2(
    caption_channels: int = 1024,
    qkv_ratio: List[float] = [0.5, 1.0],
    mlp_ratio: List[float] = [0.5, 4.0],
    pos_interp_scale: float = 1.0,
    input_size: int = 32,
    num_experts: int = 8,
    expert_capacity: float = 2.0,
    experts_every_n: int = 2,
    in_channels: int = 4,
    **kwargs
) -> DiT:
    depth = 16
    model = DiT(
        input_size=input_size,
        patch_size=2,
        in_channels=in_channels,
        dim=512,
        depth=depth,
        head_dim=32,
        multiple_of=256,
        caption_channels=caption_channels,
        pos_interp_scale=pos_interp_scale,
        norm_eps=1e-6,
        depth_init=True,
        qkv_multipliers=np.linspace(qkv_ratio[0], qkv_ratio[1], num=depth, dtype=float),
        ffn_multipliers=np.linspace(mlp_ratio[0], mlp_ratio[1], num=depth, dtype=float),
        use_patch_mixer=True,
        patch_mixer_depth=4,
        patch_mixer_dim=512,  # allocating higher budget to mixer layers
        patch_mixer_qkv_ratio=1.0,
        patch_mixer_mlp_ratio=4.0,
        use_bias=False,
        num_experts=num_experts,
        expert_capacity=expert_capacity,
        experts_every_n=experts_every_n,
        **kwargs
    )
    return model

In [ ]:
def MicroDiT_XL_2(
    caption_channels: int = 1024,
    qkv_ratio: List[float] = [0.5, 1.0],
    mlp_ratio: List[float] = [0.5, 4.0],
    pos_interp_scale: float = 1.0,
    input_size: int = 32,
    num_experts: int = 8,
    expert_capacity: float = 2.0,
    experts_every_n: int = 2,
    in_channels: int = 4,
    **kwargs
) -> DiT:
    depth = 28
    model = DiT(
        input_size=input_size,
        patch_size=2,
        in_channels=in_channels,
        dim=1024,
        depth=depth,
        head_dim=64,
        multiple_of=256,
        caption_channels=caption_channels,
        pos_interp_scale=pos_interp_scale,
        norm_eps=1e-6,
        depth_init=True,
        qkv_multipliers=np.linspace(qkv_ratio[0], qkv_ratio[1], num=depth, dtype=float),
        ffn_multipliers=np.linspace(mlp_ratio[0], mlp_ratio[1], num=depth, dtype=float),
        use_patch_mixer=True,
        patch_mixer_depth=6,
        patch_mixer_dim=768,  # allocating higher budget to mixer layers
        patch_mixer_qkv_ratio=1.0,
        patch_mixer_mlp_ratio=4.0,
        use_bias=False,
        num_experts=num_experts,
        expert_capacity=expert_capacity,
        experts_every_n=experts_every_n,
        **kwargs
    )
    return model

## LatentDiffusion

### UniversalTokenizer

In [ ]:
class simple_2_hf_tokenizer_wrapper:
    """Simple wrapper to make OpenCLIP tokenizer match HuggingFace interface.
    
    Args:
        tokenizer (Any): OpenCLIP tokenizer instance
    """
    def __init__(self, tokenizer: Any):
        self.tokenizer = tokenizer
        self.model_max_length = self.tokenizer.context_length
        
    def __call__(
        self,
        caption: str,
        padding: str = 'max_length',
        max_length: Optional[int] = None,
        truncation: bool = True,
        **kwargs
    ) -> Dict[str, torch.Tensor]:
        return {'input_ids': self.tokenizer(caption, context_length=max_length)}

In [ ]:
class UniversalTokenizer:
    """Universal tokenizer supporting multiple model types.
    
    Args:
        name (str): Name/path of the tokenizer to load
    """
    def __init__(self, name: str):
        self.name = name
        s, d = text_encoder_embedding_format(name)
        if self.name.startswith("openclip:"):
            self.tokenizer = simple_2_hf_tokenizer_wrapper(
                open_clip.get_tokenizer(name.lstrip('openclip:'))
            )
            assert s == self.tokenizer.model_max_length, "simply check of text_encoder_embedding_format"
        elif self.name == "DeepFloyd/t5-v1_1-xxl":
            self.tokenizer = T5Tokenizer.from_pretrained(name) # for t5 we would use a smaller than max_seq_length
        else:
            self.tokenizer = CLIPTokenizer.from_pretrained(name, subfolder='tokenizer')
            assert s == self.tokenizer.model_max_length, "simply check of text_encoder_embedding_format"
        self.model_max_length = s
        
    def tokenize(self, captions: Union[str, List[str]]) -> Dict[str, torch.Tensor]:
        if self.name == "DeepFloyd/t5-v1_1-xxl":
            text_tokens_and_mask = self.tokenizer(
                captions,
                padding='max_length',
                max_length=self.model_max_length,
                truncation=True,
                return_attention_mask=True,
                add_special_tokens=True,
                return_tensors='pt'
            )
            return {
                'input_ids': text_tokens_and_mask['input_ids'],
                'attention_mask': text_tokens_and_mask['attention_mask']
            }
        else:
            # Avoid attention mask for CLIP tokenizers as they are not used
            tokenized_caption = self.tokenizer(
                captions,
                padding='max_length',
                max_length=self.tokenizer.model_max_length,
                truncation=True,
                return_tensors='pt'
            )['input_ids']
            return {'input_ids': tokenized_caption}

### UniversalTextEncoder

In [ ]:
class openclip_text_encoder(torch.nn.Module):
    """OpenCLIP text encoder wrapper.
    
    Args:
        clip_model (Any): OpenCLIP model instance
        dtype (torch.dtype, torch.float32): Data type for model weights
    """
    def __init__(self, clip_model: Any, dtype: torch.dtype = torch.float32, **kwargs) -> None:
        super().__init__()
        self.clip_model = clip_model
        self.device = None
        self.dtype = dtype

    def forward_fxn(self, text: torch.Tensor) -> Tuple[torch.Tensor, None]:
        cast_dtype = self.clip_model.transformer.get_cast_dtype()
        x = self.clip_model.token_embedding(text).to(cast_dtype)  # [batch_size, n_ctx, d_model]
        x = x + self.clip_model.positional_embedding.to(cast_dtype)
        x = x.permute(1, 0, 2)  # NLD -> LND

        # アテンションマスクを外す
        # x = self.clip_model.transformer(x, attn_mask=self.clip_model.attn_mask)
        x = self.clip_model.transformer(x)
        x = x.permute(1, 0, 2)  # LND -> NLD
        x = self.clip_model.ln_final(x)  # [batch_size, n_ctx, transformer.width]
        x = x.unsqueeze(dim=1) # [batch_size, 1, n_ctx, transformer.width] expected for text_emb
        return x, None # HF encoders expected to return multiple values with first being text emb

    def forward(self, text: torch.Tensor, **kwargs) -> Tuple[torch.Tensor, None]:
        with torch.autocast(device_type='cuda', dtype=self.dtype):
            return self.forward_fxn(text)

In [ ]:
class UniversalTextEncoder(torch.nn.Module):
    """Universal text encoder supporting multiple model types.
    
    Args:
        name (str): Name/path of the model to load
        dtype (str): Data type for model weights
        pretrained (bool, True): Whether to load pretrained weights
    """
    def __init__(self, name: str, dtype: str, pretrained: bool = True):
        super().__init__()
        self.name = name
        if self.name.startswith("openclip:"):
            assert pretrained, 'Load default pretrained model from openclip'
            self.encoder = openclip_text_encoder(
                open_clip.create_model_and_transforms(name.lstrip('openclip:'))[0],
                torch_dtype=DATA_TYPES[dtype]
            )
        elif self.name == "DeepFloyd/t5-v1_1-xxl":
            self.encoder = T5EncoderModel.from_pretrained(
                name,
                torch_dtype=DATA_TYPES[dtype],
                pretrained=pretrained
            )
        else:
            self.encoder = CLIPTextModel.from_pretrained(
                name,
                subfolder='text_encoder',
                torch_dtype=DATA_TYPES[dtype],
                pretrained=pretrained
            )

    def encode(self, tokenized_caption: torch.Tensor, attention_mask: Optional[torch.Tensor] = None) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        if self.name == "DeepFloyd/t5-v1_1-xxl":
            out = self.encoder(
                tokenized_caption,
                attention_mask=attention_mask
            )['last_hidden_state']
            out = out.unsqueeze(dim=1)
            return out, None
        else:
            return self.encoder(tokenized_caption)


### DistLoss

In [ ]:
class DistLoss(Metric):
    """Distributed loss metric.
    
    Args:
        kwargs (Any): Additional arguments passed to parent class
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_state("loss", default=torch.tensor(0.), dist_reduce_fx="sum")
        self.add_state("batches", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, value: torch.Tensor) -> None:
        self.loss += value
        self.batches += 1

    def compute(self) -> torch.Tensor:
        return self.loss.float() / self.batches

### LatentDiffusion

In [ ]:
class LatentDiffusion(ComposerModel):
    """Latent diffusion model that generates images from text prompts.

    This model combines a DiT (Diffusion Transformer) model for denoising image latents,
    a VAE for encoding/decoding images to/from the latent space, and a text encoder
    for converting text prompts into embeddings. It implements the EDM (Elucidated
    Diffusion Model) sampling process.

    Args:
        dit (nn.Module): Diffusion Transformer model
        vae (AutoencoderKL): VAE model from diffusers for encoding/decoding images
        text_encoder (UniversalTextEncoder): Text encoder for converting prompts to embeddings
        tokenizer (UniversalTokenizer): Tokenizer for processing text prompts
        image_key (str, optional): Key for image data in batch dict. Defaults to 'image'.
        text_key (str, optional): Key for text data in batch dict. Defaults to 'captions'.
        image_latents_key (str, optional): Key for precomputed image latents in batch dict. Defaults to 'image_latents'.
        text_latents_key (str, optional): Key for precomputed text latents in batch dict. Defaults to 'caption_latents'.
        precomputed_latents (bool, optional): Whether to use precomputed latents (must be in the batch). Defaults to True.
        dtype (str, optional): Data type for model ops. Defaults to 'bfloat16'.
        latent_res (int, optional): Resolution of latent space assuming 8x downsampling by VAE. Defaults to 32.
        p_mean (float, optional): EDM log-normal noise mean. Defaults to -0.6.
        p_std (float, optional): EDM log-normal noise standard-deviation. Defaults to 1.2.
        train_mask_ratio (float, optional): Ratio of patches to mask during training. Defaults to 0.
    """

    def __init__(
        self,
        dit: nn.Module,
        vae: AutoencoderKL,
        text_encoder: UniversalTextEncoder,
        tokenizer: UniversalTokenizer,
        image_key: str = 'image',
        text_key: str = 'captions',
        image_latents_key: str = 'image_latents',
        text_latents_key: str = 'caption_latents',
        precomputed_latents: bool = True,
        dtype: str = 'bfloat16',
        latent_res: int = 32,
        p_mean: float = -0.6,
        p_std: float = 1.2,
        train_mask_ratio: float = 0.
    ):
        super().__init__()
        self.dit = dit
        self.vae = vae
        self.image_key = image_key
        self.text_key = text_key
        self.image_latents_key = image_latents_key
        self.text_latents_key = text_latents_key
        self.precomputed_latents = precomputed_latents
        self.dtype = dtype
        self.latent_res = latent_res
        self.edm_config = EasyDict({
            'sigma_min': 0.002,
            'sigma_max': 80,
            'P_mean': p_mean,
            'P_std': p_std,
            'sigma_data': 0.9,
            'num_steps': 18,
            'rho': 7,
            'S_churn': 0,
            'S_min': 0,
            'S_max': float('inf'),
            'S_noise': 1
        })
        self.train_mask_ratio = train_mask_ratio
        self.eval_mask_ratio = 0.  # no masking during sampling/evaluation
        assert self.train_mask_ratio >= 0, 'Masking ratio must be non-negative!'

        self.randn_like = torch.randn_like
        self.latent_scale = self.vae.config.scaling_factor

        self.text_encoder = text_encoder
        self.tokenizer = tokenizer
        # freeze vae and text_encoder during training
        self.text_encoder.requires_grad_(False)
        self.vae.requires_grad_(False)
        # avoid wrapping the models that we aren't training
        self.text_encoder._fsdp_wrap = False
        self.vae._fsdp_wrap = False
        self.dit._fsdp_wrap = True

    def forward(self, batch: dict) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # Get image latents
        if self.precomputed_latents and self.image_latents_key in batch:
            # Assuming that latents have already been scaled, i.e., multiplied with the scaling factor
            latents = batch[self.image_latents_key]
        else:
            with torch.no_grad():
                images = batch[self.image_key]
                latents = self.vae.encode(
                    images.to(DATA_TYPES[self.dtype])
                )['latent_dist'].sample().data
                latents *= self.latent_scale

        # Get text embeddings
        if self.precomputed_latents and self.text_latents_key in batch:
            conditioning = batch[self.text_latents_key]
        else:
            captions = batch[self.text_key]
            captions = captions.view(-1, captions.shape[-1])
            if 'attention_mask' in batch:
                conditioning = self.text_encoder.encode(
                    captions,
                    attention_mask=batch['attention_mask'].view(-1, captions.shape[-1])
                )[0]
            else:
                conditioning = self.text_encoder.encode(captions)[0]

        # Zero out dropped captions. Needed for classifier-free guidance during inference.
        if 'drop_caption_mask' in batch.keys():
            conditioning *= batch['drop_caption_mask'].view(
                [-1] + [1] * (len(conditioning.shape) - 1)
            )

        loss = self.edm_loss(
            latents.float(),
            conditioning.float(),
            mask_ratio=self.train_mask_ratio if self.training else self.eval_mask_ratio
        )
        return (loss, latents, conditioning)

    def model_forward_wrapper(
        self,
        x: torch.Tensor,
        sigma: torch.Tensor,
        y: torch.Tensor,
        model_forward_fxn: callable,
        mask_ratio: float,
        **kwargs
    ) -> dict:
        """Wrapper for the model call in EDM (https://github.com/NVlabs/edm/blob/main/training/networks.py#L632)"""
        sigma = sigma.to(x.dtype).reshape(-1, 1, 1, 1)
        c_skip = (
            self.edm_config.sigma_data ** 2 /
            (sigma ** 2 + self.edm_config.sigma_data ** 2)
        )
        c_out = (
            sigma * self.edm_config.sigma_data /
            (sigma ** 2 + self.edm_config.sigma_data ** 2).sqrt()
        )
        c_in = 1 / (self.edm_config.sigma_data ** 2 + sigma ** 2).sqrt()
        c_noise = sigma.log() / 4

        out = model_forward_fxn(
            (c_in * x).to(x.dtype),
            c_noise.flatten(),
            y,
            mask_ratio=mask_ratio,
            **kwargs
        )
        F_x = out['sample']
        c_skip = c_skip.to(F_x.device)
        x = x.to(F_x.device)
        c_out = c_out.to(F_x.device)
        D_x = c_skip * x + c_out * F_x
        out['sample'] = D_x
        return out

    def edm_loss(self, x: torch.Tensor, y: torch.Tensor, mask_ratio: float = 0, **kwargs) -> torch.Tensor:
        rnd_normal = torch.randn([x.shape[0], 1, 1, 1], device=x.device)
        sigma = (rnd_normal * self.edm_config.P_std + self.edm_config.P_mean).exp()
        weight = (
            (sigma ** 2 + self.edm_config.sigma_data ** 2) /
            (sigma * self.edm_config.sigma_data) ** 2
        )
        n = self.randn_like(x) * sigma

        model_out = self.model_forward_wrapper(
            x + n,
            sigma,
            y,
            self.dit,
            mask_ratio=mask_ratio,
            **kwargs
        )
        D_xn = model_out['sample']
        loss = weight * ((D_xn - x) ** 2)  # (N, C, H, W)

        if mask_ratio > 0:
            # Masking is not feasible during image generation as it only returns denoised version
            # for non-masked patches. Image generation requires all patches to be denoised.
            assert (
                self.dit.training and 'mask' in model_out
            ), 'Masking is only recommended during training'
            loss = F.avg_pool2d(loss.mean(dim=1), self.dit.patch_size).flatten(1)
            unmask = 1 - model_out['mask']
            loss = (loss * unmask).sum(dim=1) / unmask.sum(dim=1)  # (N,)
        return loss.mean()

    # Composer specific formatting of model loss and eval functions.
    def loss(self, outputs: tuple, batch: dict) -> torch.Tensor:
        # forward pass already computed the loss function
        return outputs[0]

    def eval_forward(self, batch: dict, outputs: Optional[tuple] = None) -> tuple:
        # Skip if output already calculated (e.g., during training forward pass)
        if outputs is not None:
            return outputs
        loss, _, _ = self.forward(batch)
        return loss, None, None

    def get_metrics(self, is_train: bool = False) -> dict:
        # get_metrics expected to return a dict in composer
        return {'loss': DistLoss()}

    def update_metric(self, batch: dict, outputs: tuple, metric: DistLoss):
        metric.update(outputs[0])

    @torch.no_grad()
    def edm_sampler_loop(
        self, x: torch.Tensor, 
        y: torch.Tensor, 
        steps: Optional[int] = None, 
        cfg: float = 1.0, 
        **kwargs
    ) -> torch.Tensor:
        mask_ratio = 0  # no masking during image generation
        model_forward_fxn = (
            partial(self.dit.forward, cfg=cfg) if cfg > 1.0
            else self.dit.forward
        )

        # Time step discretization.
        num_steps = self.edm_config.num_steps if steps is None else steps
        step_indices = torch.arange(num_steps, dtype=torch.float64, device=x.device)
        t_steps = (
            self.edm_config.sigma_max ** (1 / self.edm_config.rho) +
            step_indices / (num_steps - 1) *
            (self.edm_config.sigma_min ** (1 / self.edm_config.rho) -
             self.edm_config.sigma_max ** (1 / self.edm_config.rho))
        ) ** self.edm_config.rho
        t_steps = torch.cat([torch.as_tensor(t_steps), torch.zeros_like(t_steps[:1])])

        # Main sampling loop.
        x_next = x.to(torch.float64) * t_steps[0]
        for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])):  # 0, ..., N-1
            x_cur = x_next
            # Increase noise temporarily.
            gamma = (
                min(self.edm_config.S_churn / num_steps, np.sqrt(2) - 1)
                if self.edm_config.S_min <= t_cur <= self.edm_config.S_max else 0
            )
            t_hat = torch.as_tensor(t_cur + gamma * t_cur)
            x_hat = (
                x_cur +
                (t_hat ** 2 - t_cur ** 2).sqrt() *
                self.edm_config.S_noise *
                self.randn_like(x_cur)
            )

            # Euler step.
            denoised = self.model_forward_wrapper(
                x_hat.to(torch.float32),
                t_hat.to(torch.float32),
                y,
                model_forward_fxn,
                mask_ratio=mask_ratio,
                **kwargs
            )['sample'].to(torch.float64)
            d_cur = (x_hat - denoised) / t_hat
            x_next = x_hat + (t_next - t_hat) * d_cur

            # Apply 2nd order correction.
            if i < num_steps - 1:
                denoised = self.model_forward_wrapper(
                    x_next.to(torch.float32),
                    t_next.to(torch.float32),
                    y,
                    model_forward_fxn,
                    mask_ratio=mask_ratio,
                    **kwargs
                )['sample'].to(torch.float64)
                d_prime = (x_next - denoised) / t_next
                x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)
        return x_next.to(torch.float32)

    @torch.no_grad()
    def generate(
        self,
        prompt: Optional[list] = None,
        tokenized_prompts: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.LongTensor] = None,
        guidance_scale: Optional[float] = 5.0,
        num_inference_steps: Optional[int] = 30,
        seed: Optional[int] = None,
        return_only_latents: Optional[bool] = False,
        **kwargs
    ) -> torch.Tensor:
        # _check_prompt_given(prompt, tokenized_prompts, prompt_embeds=None)
        assert prompt or tokenized_prompts, "Must provide either prompt or tokenized prompts"
        device = self.vae.device  # hack to identify model device during training
        rng_generator = torch.Generator(device=device)
        if seed:
            rng_generator = rng_generator.manual_seed(seed)

        # Convert prompt text to embeddings (zero out embeddings for classifier-free guidance)
        if tokenized_prompts is None:
            out = self.tokenizer.tokenize(prompt)
            tokenized_prompts = out['input_ids']
            attention_mask = (
                out['attention_mask'] if 'attention_mask' in out else None
            )
        text_embeddings = self.text_encoder.encode(
            tokenized_prompts.to(device),
            attention_mask=attention_mask.to(device) if attention_mask is not None else None
        )[0]

        latents = torch.randn(
            (len(text_embeddings), self.dit.in_channels, self.latent_res, self.latent_res),
            device=device,
            generator=rng_generator,
        )

        # iteratively denoise latents
        latents = self.edm_sampler_loop(
            latents,
            text_embeddings,
            num_inference_steps,
            cfg=guidance_scale
        )

        if return_only_latents:
            return latents

        # Decode latents with VAE
        latents = 1 / self.latent_scale * latents
        torch_dtype = DATA_TYPES[self.dtype]
        image = self.vae.decode(latents.to(torch_dtype)).sample
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.float().detach()
        return image

In [ ]:
def text_encoder_embedding_format(enc: str) -> Tuple[int, int]:
    """Returns sequence length and token embedding dimension for text encoder."""
    if enc in [
        'stabilityai/stable-diffusion-2-base',
        'runwayml/stable-diffusion-v1-5',
        'CompVis/stable-diffusion-v1-4'
    ]:
        return 77, 1024
    if enc in ['openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378']:
        return 77, 1024
    if enc in ["DeepFloyd/t5-v1_1-xxl"]:
        return 120, 4096
    raise ValueError(f'Please specifcy the sequence and embedding size of {enc} encoder')

In [ ]:
def create_latent_diffusion(
    vae_name: str = 'stabilityai/stable-diffusion-xl-base-1.0',
    text_encoder_name: str = 'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378', 
    dit_arch: str = 'MicroDiT_XL_2',
    latent_res: int = 32,
    in_channels: int = 4,
    pos_interp_scale: float = 1.0,
    dtype: str = 'bfloat16',
    precomputed_latents: bool = True,
    p_mean: float = -0.6,
    p_std: float = 1.2,
    train_mask_ratio: float = 0.
) -> LatentDiffusion:
    # retrieve max sequence length (s) and token embedding dim (d) from text encoder
    s, d = text_encoder_embedding_format(text_encoder_name)

    if dit_arch == 'MicroDiT_Tiny_2':
        dit = MicroDiT_Tiny_2(
            input_size=latent_res,
            caption_channels=d,
            pos_interp_scale=pos_interp_scale,
            in_channels=in_channels
        )
    elif dit_arch == 'MicroDiT_XL_2':
        dit = MicroDiT_XL_2(
            input_size=latent_res,
            caption_channels=d,
            pos_interp_scale=pos_interp_scale,
            in_channels=in_channels
        )
    else:
        raise ValueError(f'Unknown DiT architecture: {dit_arch}')

    vae = AutoencoderKL.from_pretrained(
        vae_name,
        subfolder=None if vae_name=='ostris/vae-kl-f8-d16' else 'vae',
        torch_dtype=DATA_TYPES[dtype],
        pretrained=True
    )

    text_encoder = UniversalTextEncoder(
        text_encoder_name,
        dtype=dtype,
        pretrained=True
    )
    tokenizer = UniversalTokenizer(text_encoder_name)

    model = LatentDiffusion(
        dit=dit,
        vae=vae,
        text_encoder=text_encoder,
        tokenizer=tokenizer,
        precomputed_latents=precomputed_latents,
        dtype=dtype,
        latent_res=latent_res,
        p_mean=p_mean,
        p_std=p_std,
        train_mask_ratio=train_mask_ratio
    )
    return model

## データセット構築

### ダウンロード

In [ ]:
def download_and_process_metadata():
    # Only using a single process for downloading metadata
    metadata_files = [
        ('data/train', 'train_anno.jsonl.tgz'),
        ('data/train', 'train_anno_realease_repath.jsonl.tgz'),
        ('data/valid', 'valid_anno_repath.jsonl.tgz'),
        ('data/test', 'test_questions.jsonl.tgz'),
        ('data/test', 'imgs.tgz'),
    ]

    for subfolder, filename in metadata_files:
        hf_hub_download(
            repo_id="JourneyDB/JourneyDB",
            repo_type="dataset",
            subfolder=subfolder,
            filename=filename,
            local_dir=COMPRESSED_DIR,
            local_dir_use_symlinks=False,
        )

    metadata_tars = [
        os.path.join(dir, fname) for (dir, fname) in metadata_files
    ]

    for tar_file in metadata_tars:
        subprocess.call(
            f'tar -xvzf {os.path.join(COMPRESSED_DIR, tar_file)} '
            f'-C {os.path.join(COMPRESSED_DIR, os.path.dirname(tar_file))}',
            shell=True,
        )

    shutil.copy(
        f'{os.path.join(COMPRESSED_DIR, "data/train/train_anno_realease_repath.jsonl")}',
        f'{os.path.join(RAW_DIR, "train/train_anno_realease_repath.jsonl")}',
    )

    shutil.copy(
        f'{os.path.join(COMPRESSED_DIR, "data/valid/valid_anno_repath.jsonl")}',
        f'{os.path.join(RAW_DIR, "valid/valid_anno_repath.jsonl")}',
    )

    shutil.copy(
        f'{os.path.join(COMPRESSED_DIR, "data/test/test_questions.jsonl")}',
        f'{os.path.join(RAW_DIR, "test/test_questions.jsonl")}',
    )

    shutil.move(
        f'{os.path.join(COMPRESSED_DIR, "data/test/imgs")}',
        f'{os.path.join(RAW_DIR, "test/")}',
    )

COMPRESSED_DIR = os.path.join(DATA_DIR, 'compressed')
logger.info(f"{COMPRESSED_DIR=}")

RAW_DIR = os.path.join(DATA_DIR, 'raw')
logger.info(f"{RAW_DIR=}")

if not os.path.exists(COMPRESSED_DIR):
    os.makedirs(COMPRESSED_DIR, exist_ok=True)
    os.makedirs(RAW_DIR, exist_ok=True)
    download_and_process_metadata()

### 解凍とリサイズ

In [ ]:
def download_uncompress_resize(
    valid_ids: list,
    max_image_size: int,
    min_image_size: int,
    split: str,
    idx: int,
):
    """Download, uncompress, and resize images for a given archive index."""
    assert split in ('train', 'valid')
    assert idx in valid_ids

    print(f"Downloading idx: {idx}")
    if not os.path.exists(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}/'):
        hf_hub_download(
            repo_id="JourneyDB/JourneyDB",
            repo_type="dataset",
            subfolder=f'data/{split}/imgs',
            filename=f'{idx:>03}.tgz',
            local_dir=COMPRESSED_DIR,
            local_dir_use_symlinks=False,
        )
    print(f"Downloaded idx: {idx}")

    print(f"Extracting idx: {idx}")
    if not os.path.exists(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}/'):
        subprocess.call(
            f'tar -xzf {COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}.tgz '
            f'-C {COMPRESSED_DIR}/data/{split}/imgs/',
            shell=True,
        )
    print(f"Extracted idx: {idx}")

    print(f"Removing idx: {idx}")
    if os.path.exists(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}.tgz'):
        os.remove(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}.tgz')
    print(f"Removed idx: {idx}")

    # add bicubic downsize
    downsize = transforms.Resize(
        max_image_size,
        antialias=True,
        interpolation=transforms.InterpolationMode.BICUBIC,
    )

    print(f"Downsizing idx: {idx}")
    os.makedirs(
        f'{RAW_DIR}/{split}/imgs/{idx:>03}/',
        exist_ok=True,
    )
    for f in iglob(f'{COMPRESSED_DIR}/data/{split}/imgs/{idx:>03}/*'):
        save_path = f'{RAW_DIR}/{split}/imgs/{idx:>03}/{os.path.basename(f)}'

        if os.path.exists(save_path):
            continue

        if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            try:
                img = Image.open(f)
                w, h = img.size
                if min(w, h) > max_image_size:
                    img = downsize(img)
                if min(w, h) < min_image_size:
                    print(
                        f'Skipping image with resolution ({h}, {w}) - '
                        f'Since at least one side has resolution below {min_image_size}'
                    )
                    continue

                img.save(save_path)
                os.remove(f)
            except (UnidentifiedImageError, OSError) as e:
                print(f"Error {e}, File: {f}")
    print(f'Downsized idx: {idx}')


if not os.path.exists(os.path.join(RAW_DIR, 'train/imgs')):

    valid_ids = list(np.arange(1))
    pool_args = [('train', i) for i in valid_ids] + [('valid', i) for i in valid_ids]
    max_image_size = 512
    min_image_size = 64

    with Pool(processes=4) as pool:
        pool.starmap(
            download_uncompress_resize,
            [(
                valid_ids,
                max_image_size,
                min_image_size,
                split,
                idx
            ) for split, idx in pool_args]
        )

### MDS形式に変換

In [ ]:
def convert_to_mds(
    images_dir: str,
    captions_jsonl: str,
    local_mds_dir: str,
):
    """
    JourneyDBをMDS形式に変換する
    MDS（Mosaic Data Storage）は、大規模データセット用の高速ストリーミングフォーマット

    Args:
        images_dir: 画像ディレクトリのパス
        captions_jsonl: キャプションのJSONLファイルのパス
        local_mds_dir: 出力MDSディレクトリのパス
    """

    columns = {
        'width': 'int32',
        'height': 'int32',
        'jpg': 'jpeg',
        'caption': 'str',
    }
    
    writer = MDSWriter(
        out=local_mds_dir,
        columns=columns,
        compression=None,
        size_limit=256 * (2**20),
        max_workers=64,
    )
    
    # Retrieving achieve indies, in case only a subset of the data is downloaded
    valid_archieve_idx = [
        os.path.basename(p) for p in glob(os.path.join(images_dir, '*'))
    ]
    
    metadata = list(open(captions_jsonl, 'r'))

    for f in tqdm(metadata):
        d = json.loads(f)
        cap, p = d['prompt'], d['img_path'].strip('./')
        
        if os.path.dirname(p) not in valid_archieve_idx:
            continue
            
        try:
            img = Image.open(os.path.join(images_dir, p))
            w, h = img.size
            mds_sample = {
                'jpg': img,
                'caption': cap,
                'width': w,
                'height': h,
            }
            writer.write(mds_sample)
        except Exception as e:
            print(
                "Something went wrong in reading caption, "
                f"skipping writing this sample in mds. Error: {e}"
            )

    writer.finish()

MDS_DIR = os.path.join(DATA_DIR, 'jdb', 'mds')
logger.info(f"{MDS_DIR=}")

if not os.path.exists(MDS_DIR):
    convert_to_mds(
        images_dir=os.path.join(RAW_DIR, 'train', 'imgs'),
        captions_jsonl=os.path.join(RAW_DIR, 'train', 'train_anno_realease_repath.jsonl'),
        local_mds_dir=os.path.join(MDS_DIR, 'train'),
    )

    convert_to_mds(
        images_dir=os.path.join(RAW_DIR, 'valid', 'imgs'),
        captions_jsonl=os.path.join(RAW_DIR, 'valid', 'valid_anno_repath.jsonl'),
        local_mds_dir=os.path.join(MDS_DIR, 'valid'),
    )

### 前処理

In [ ]:
class StreamingJdbDatasetForPreCompute(StreamingDataset):
    """Streaming dataset that resizes images to user-provided resolutions and tokenizes captions."""

    def __init__(
        self,
        streams: Sequence[Stream],
        transforms_list: List[Callable],
        batch_size: int,
        tokenizer_name: str,
        shuffle: bool = False,
        caption_key: str = 'caption',
    ):
        super().__init__(
            streams=streams,
            shuffle=shuffle,
            batch_size=batch_size,
        )

        self.transforms_list = transforms_list
        self.caption_key = caption_key
        self.tokenizer = UniversalTokenizer(tokenizer_name)
        print("Created tokenizer: ", tokenizer_name)
        assert self.transforms_list is not None, 'Must provide transforms to resize and center crop images'

    def __getitem__(self, index: int) -> Dict:
        sample = super().__getitem__(index)
        ret = {}

        out = self.tokenizer.tokenize(sample[self.caption_key])
        ret[self.caption_key] = out['input_ids'].clone().detach()
        if 'attention_mask' in out:
            ret[f'{self.caption_key}_attention_mask'] = out['attention_mask'].clone().detach()

        for i, tr in enumerate(self.transforms_list):
            img = sample['jpg']
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img = tr(img)
            ret[f'image_{i}'] = img

        ret['sample'] = sample
        return ret

In [ ]:
def build_streaming_jdb_precompute_dataloader(
    datadir: Union[List[str], str],
    batch_size: int,
    resize_sizes: Optional[List[int]] = None,
    drop_last: bool = False,
    shuffle: bool = True,
    caption_key: Optional[str] = None,
    tokenizer_name: Optional[str] = None,
    **dataloader_kwargs,
) -> DataLoader:
    """Builds a streaming mds dataloader returning multiple image sizes and text captions."""
    assert resize_sizes is not None, 'Must provide target resolution for image resizing'
    datadir = [datadir] if isinstance(datadir, str) else datadir
    streams = [Stream(remote=None, local=l) for l in datadir]

    transforms_list = []
    for resize in resize_sizes:
        transforms_list.append(
            transforms.Compose([
                transforms.Resize(
                    resize,
                    interpolation=transforms.InterpolationMode.BICUBIC,
                ),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])
        )

    dataset = StreamingJdbDatasetForPreCompute(
        streams=streams,
        shuffle=shuffle,
        transforms_list=transforms_list,
        batch_size=batch_size,
        caption_key=caption_key,
        tokenizer_name=tokenizer_name,
    )

    def custom_collate(list_of_dict: List[Dict]) -> Dict:
        out = {k: [] for k in list_of_dict[0].keys()}
        for d in list_of_dict:
            for k, v in d.items():
                out[k].append(v)
        return out

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        drop_last=drop_last,
        collate_fn=custom_collate,
        **dataloader_kwargs,
    )

    return dataloader

In [ ]:
def precompute(
    datadir: str,
    savedir: str = "",
    image_resolutions: list = [256, 512],
    save_images: bool = False,
    model_dtype: str = "bfloat16",
    save_dtype: str = "float16",
    vae: str = "stabilityai/stable-diffusion-xl-base-1.0",
    text_encoder: str = "openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378",
    batch_size: int = 32,
    seed: int = 2024,
):
    """Precompute image and text latents and store them in MDS format.

    By default, we only save the image latents for 256x256 and 512x512 image
    resolutions (using center crop).

    Note that the image latents will be scaled by the vae_scaling_factor.
    """
    cap_key = 'caption'  # Hardcoding the image caption key to 'caption' in MDS dataset

    accelerator = Accelerator()
    device = accelerator.device
    device_idx = int(accelerator.process_index)

    # Set random seeds
    torch.manual_seed(device_idx + seed)
    torch.cuda.manual_seed(device_idx + seed)
    np.random.seed(device_idx + seed)

    dataloader = build_streaming_jdb_precompute_dataloader(
        datadir=[datadir],
        batch_size=batch_size,
        resize_sizes=image_resolutions,
        drop_last=False,
        shuffle=False,
        caption_key=cap_key,
        tokenizer_name=text_encoder,
        # prefetch_factor=2,
        # num_workers=2,
        # persistent_workers=True,
        pin_memory=True,
    )
    print(f'Device: {device_idx}, Dataloader sample count: {len(dataloader.dataset)}')

    # print(
    #     f"MP variable -> world size: {os.environ['WORLD_SIZE']}, "
    #     f"RANK: {os.environ['RANK']}, {device}"
    # )

    vae = AutoencoderKL.from_pretrained(
        vae,
        subfolder='vae',  # Change subfolder to appropriate one in hf_hub, if needed
        torch_dtype=DATA_TYPES[model_dtype],
    )
    print("Created VAE: ", vae)
    assert isinstance(vae, AutoencoderKL)

    text_encoder = UniversalTextEncoder(
        text_encoder,
        dtype=model_dtype,
        pretrained=True,
    )
    print("Created text encoder: ", text_encoder)

    vae = vae.to(device)
    text_encoder = text_encoder.to(device)

    columns = {
        cap_key: 'str',
        f'{cap_key}_latents': 'bytes',
        'latents_256': 'bytes',
        'latents_512': 'bytes',
    }
    if save_images:
        columns['jpg'] = 'jpeg'

    remote_upload = os.path.join(savedir, str(accelerator.process_index))

    writer = MDSWriter(
        out=remote_upload,
        columns=columns,
        compression=None,
        size_limit=256 * (2**20),
        max_workers=64,
    )

    for batch in tqdm(dataloader):
        image_256 = torch.stack(batch['image_0']).to(device)
        image_512 = torch.stack(batch['image_1']).to(device)
        captions = torch.stack(batch[cap_key]).to(device)

        with torch.no_grad():
            with torch.autocast(device_type='cuda', dtype=DATA_TYPES[model_dtype]):
                latent_dist_256 = vae.encode(image_256)
                assert isinstance(latent_dist_256, AutoencoderKLOutput)
                latents_256 = (
                    latent_dist_256['latent_dist'].sample().data * vae.config.scaling_factor
                ).to(DATA_TYPES[save_dtype])

                latent_dist_512 = vae.encode(image_512)
                assert isinstance(latent_dist_512, AutoencoderKLOutput)
                latents_512 = (
                    latent_dist_512['latent_dist'].sample().data * vae.config.scaling_factor
                ).to(DATA_TYPES[save_dtype])

                attention_mask = None

                if f'{cap_key}_attention_mask' in batch:
                    attention_mask = torch.stack(
                        batch[f'{cap_key}_attention_mask']
                    ).to(device)

                conditioning = text_encoder.encode(
                    captions.view(-1, captions.shape[-1]),
                    attention_mask=attention_mask,
                )[0].to(DATA_TYPES[save_dtype])

        try:
            if isinstance(latents_256, torch.Tensor) and isinstance(
                latents_512, torch.Tensor
            ):
                latents_256 = latents_256.detach().cpu().numpy()
                latents_512 = latents_512.detach().cpu().numpy()
            else:
                continue

            if isinstance(conditioning, torch.Tensor):
                conditioning = conditioning.detach().cpu().numpy()
            else:
                continue

            # Write the batch to the MDS file
            for i in range(latents_256.shape[0]):
                mds_sample = {
                    cap_key: batch['sample'][i][cap_key],
                    f'{cap_key}_latents': np.reshape(conditioning[i], -1).tobytes(),
                    'latents_256': latents_256[i].tobytes(),
                    'latents_512': latents_512[i].tobytes(),
                }
                if save_images:
                    mds_sample['jpg'] = batch['sample'][i]['jpg']
                writer.write(mds_sample)
        except RuntimeError:
            print('Runtime error CUDA, skipping this batch')

    writer.finish()

    # Wait for all processes to finish
    accelerator.wait_for_everyone()
    print(f'Process {accelerator.process_index} finished')
    time.sleep(10)

    # Merge the mds shards created by each device (only do on main process)
    if accelerator.is_main_process:
        shards_metadata = [
            os.path.join(savedir, str(i), 'index.json')
            for i in range(accelerator.num_processes)
        ]
        merge_index(shards_metadata, out=savedir, keep_local=True)

PRECOMPUTE_DIR = os.path.join(DATA_DIR, 'mds_latents_sdxl1_dfnclipH14',)
logger.info(f"{PRECOMPUTE_DIR=}")

if not os.path.exists(PRECOMPUTE_DIR):
    precompute(
        datadir=os.path.join(DATA_DIR, 'jdb', 'mds', 'train'),
        savedir=os.path.join(PRECOMPUTE_DIR, 'train'),
        image_resolutions=[256, 512],
        save_images=False,
        model_dtype="bfloat16",
        save_dtype="float16",
        vae="stabilityai/stable-diffusion-xl-base-1.0",
        text_encoder="openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378",
        batch_size=16,
        seed=2024,
    )

In [ ]:
assert os.path.exists(os.path.join(PRECOMPUTE_DIR, 'train', 'index.json'))

## 訓練

### ステップ１

In [ ]:
model = create_latent_diffusion(
    vae_name='stabilityai/stable-diffusion-xl-base-1.0',
    text_encoder_name='openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378',
    # dit_arch='MicroDiT_XL_2',
    dit_arch='MicroDiT_Tiny_2',
    latent_res=32,
    in_channels=4,
    pos_interp_scale=1.0,
    dtype='bfloat16',
    precomputed_latents=True,
    p_mean=-0.6,
    p_std=1.2,
    train_mask_ratio=0.75
)

In [ ]:
# Set up optimizer with special handling for MoE parameters

moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]

if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[{'params': rest_params}, {'params': moe_params, 'lr': cfg.optimizer.lr / 2}], **opt_dict)
else:
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=2.4e-4,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

In [ ]:
# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)
logger.info(f"{cap_seq_size=}, {cap_emb_dim=}")

In [ ]:
class StreamingLatentsDataset(StreamingDataset):
    """Dataset class for loading precomputed latents from mds format.
    
    Args:
        streams: List of individual streams (in our case streams of individual datasets)
        shuffle: Whether to shuffle the dataset
        image_size: Size of images (256 or 512)
        cap_seq_size: Context length of text-encoder
        cap_emb_dim: Dimension of caption embeddings
        cap_drop_prob: Probability of using all zeros caption embedding (classifier-free guidance)
        batch_size: Batch size for streaming
    """

    def __init__(
        self,
        streams: Optional[List[Stream]] = None,
        shuffle: bool = False,
        image_size: Optional[int] = None,
        cap_seq_size: Optional[int] = None,
        cap_emb_dim: Optional[int] = None,
        cap_drop_prob: float = 0.0,
        batch_size: Optional[int] = None,
        **kwargs
    ) -> None:
        super().__init__(
            streams=streams,
            shuffle=shuffle,
            batch_size=batch_size,
        )

        self.image_size = image_size
        self.cap_seq_size = cap_seq_size
        self.cap_emb_dim = cap_emb_dim
        self.cap_drop_prob = cap_drop_prob

    def __getitem__(self, index: int) -> Dict[str, Union[torch.Tensor, str, float]]:
        sample = super().__getitem__(index)
        out = {}

        # Mask for zero'ed out captions in classifier-free guidance (cfg) training.
        # We replace caption embeddings with a zero vector in cfg guidance.
        out['drop_caption_mask'] = (
            0. if torch.rand(1) < self.cap_drop_prob else 1.
        )
        out['caption_latents'] = torch.from_numpy(
            np.frombuffer(sample['caption_latents'], dtype=np.float16)
            .copy()
        ).reshape(1, self.cap_seq_size, self.cap_emb_dim)

        if self.image_size == 256 and 'latents_256' in sample:
            out['image_latents'] = torch.from_numpy(
                np.frombuffer(sample['latents_256'], dtype=np.float16)
                .copy()
            ).reshape(-1, 32, 32)

        if self.image_size == 512 and 'latents_512' in sample:
            out['image_latents'] = torch.from_numpy(
                np.frombuffer(sample['latents_512'], dtype=np.float16)
                .copy()
            ).reshape(-1, 64, 64)

        # out['caption'] = sample['caption']
        return out

In [ ]:
def build_streaming_latents_dataloader(
    datadir: Union[str, List[str]],
    batch_size: int,
    image_size: int = 256,
    cap_seq_size: int = 77,
    cap_emb_dim: int = 1024,
    cap_drop_prob: float = 0.0,
    shuffle: bool = True,
    drop_last: bool = True,
    **dataloader_kwargs
) -> DataLoader:
    """Creates a DataLoader for streaming latents dataset."""
    if isinstance(datadir, str):
        datadir = [datadir]

    streams = [Stream(remote=None, local=d) for d in datadir]

    dataset = StreamingLatentsDataset(
        streams=streams,
        shuffle=shuffle,
        image_size=image_size,
        cap_seq_size=cap_seq_size,
        cap_emb_dim=cap_emb_dim,
        cap_drop_prob=cap_drop_prob,
        batch_size=batch_size,
    )

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        sampler=None,
        drop_last=drop_last,
        **dataloader_kwargs,
    )

    return dataloader

In [ ]:
train_loader = build_streaming_latents_dataloader(
    datadir=['/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'],
    image_size=256,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.1,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

num_train_dataset_images = len(train_loader.dataset) * dist.get_world_size()
logger.info(f"{num_train_dataset_images=}")

In [ ]:
eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=256,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

num_eval_dataset_images = len(eval_loader.dataset) * dist.get_world_size()
logger.info(f"{num_eval_dataset_images=}")

In [ ]:
loggers = []
loggers.append(TensorboardLogger())

exp_name = 'MicroDiTXL_mask_75_res_256_pretrain'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)
# loggers.append(wandb_logger)

In [ ]:
apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)

algorithms = []
algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)

callbacks.append(NaNCatcher())

In [ ]:
scheduler = CosineAnnealingWithWarmupScheduler(
    t_warmup='2500ba',
    alpha_f=0.33 # decay to 0.8e-4 after 256x256 masked pre-training
)

In [ ]:
# torch.compileにより15%程度高速化
compile = False

trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='250000ba',
    max_duration='1ba',
    eval_interval='2500ba' if compile else 0,
    save_interval='2500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=256,
    run_name='microdit_experiment',
    seed=42,
    save_folder='./trained_models/microdit_experiment/',
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None
)

In [ ]:
 # Ensure models are on correct device
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
logger.setLevel(logging.WARNING)  # to reduce streaming dataset info logs
trainer.fit()

### ステップ２

In [ ]:
# Set up optimizer with special handling for MoE parameters
moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]

if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[
            {'params': rest_params},
            {'params': moe_params, 'lr': 8e-5 / 2}], **opt_dict)
else:
    # optimizer = hydra.utils.instantiate(cfg.optimizer, params=model.dit.parameters())
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=8e-5,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

In [ ]:
# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
# Set up data loaders

cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)

train_loader = build_streaming_latents_dataloader(
    datadir=[
        '/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'
    ],
    image_size=256,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.1,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=256,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)


In [ ]:
loggers = []
loggers.append(TensorboardLogger())

exp_name = 'MicroDiTXL_mask_0_res_256_finetune'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)

In [ ]:
algorithms = []

apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)

algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)
callbacks.append(NaNCatcher())

In [ ]:
scheduler = ConstantScheduler(alpha=1.0)

In [ ]:
compile = False

fsdp_config = {
    'sharding_strategy': 'SHARD_GRAD_OP',
}

trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='280000ba',
    max_duration='1ba',
    eval_interval='2500ba',
    save_interval='2500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=64,
    run_name='microdit_experiment',
    seed=18,
    save_folder='./trained_models/microdit_experiment/',
    # load_path='./trained_models/MicroDiTXL_mask_75_res_256_pretrain/latest-rank0.pt',
    # load_ignore_keys=[
    #     "state/optimizers/AdamW/param_groups/initial_lr",
    #     "state/optimizers/AdamW/param_groups/lr",
    #     "state/schedulers/LambdaLR/base_lrs",
    #     "state/schedulers/LambdaLR/_last_lr"
    # ],
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None,
    # fsdp_config=fsdp_config
)

In [ ]:
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
trainer.fit()

### ステップ３

In [ ]:
# Set up optimizer with special handling for MoE parameters

moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]

if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[
            {'params': rest_params},
            {'params': moe_params, 'lr': 8e-5 / 2}], **opt_dict)
else:
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=8e-5,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

In [ ]:
# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
# Set up data loaders
# cap_seq_size, cap_emb_dim = text_encoder_embedding_format(cfg.model.text_encoder_name)
cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)

train_loader = build_streaming_latents_dataloader(
    datadir=[
        '/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'
    ],
    image_size=256,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.0,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

# print(f"Found {len(train_loader.dataset)*dist.get_world_size()} images in the training dataset")
# time.sleep(3)

eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=256,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

In [ ]:
loggers = []
loggers.append(TensorboardLogger())
exp_name = 'MicroDiTXL_mask_0_res_256_finetune_final'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)
# loggers.append(wandb_logger)

In [ ]:
algorithms = []
apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)
algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)
callbacks.append(NaNCatcher())

In [ ]:
scheduler = ConstantScheduler(alpha=1.0)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='50000ba',
    max_duration='1ba',
    eval_interval='2500ba',
    save_interval='2500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=64,
    run_name='microdit_experiment',
    seed=18,
    save_folder='./trained_models/microdit_experiment/',
    load_path='./trained_models/MicroDiTXL_mask_0_res_256_finetune/latest-rank0.pt',
    load_weights_only=True,
    load_strict_model_weights=False,
    load_ignore_keys=["state/model/dit.pos_embed"],
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None,
    # fsdp_config=fsdp_config
)

In [ ]:
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
trainer.fit()

### ステップ４

In [ ]:
# Set up optimizer with special handling for MoE parameters
moe_params = [p[1] for p in model.dit.named_parameters() if 'moe' in p[0].lower()]
rest_params = [p[1] for p in model.dit.named_parameters() if 'moe' not in p[0].lower()]
if len(moe_params) > 0:
    print('Reducing learning rate of MoE parameters by 1/2')
    opt_dict = dict(cfg.optimizer)
    opt_name = opt_dict['_target_'].split('.')[-1]
    del opt_dict['_target_']
    optimizer = getattr(torch.optim, opt_name)(
        params=[{'params': rest_params}, {'params': moe_params, 'lr': cfg.optimizer.lr / 2}], **opt_dict)
else:
    # optimizer = hydra.utils.instantiate(cfg.optimizer, params=model.dit.parameters())
    optimizer = torch.optim.AdamW(
        params=model.dit.parameters(),
        lr=8e-5,
        weight_decay=0.1,
        eps=1.0e-8,
        betas=(0.9, 0.999)
    )

# Convert ListConfig betas to native list to avoid ValueError when saving optimizer state
for p in optimizer.param_groups:
    p['betas'] = list(p['betas'])

In [ ]:
cap_seq_size, cap_emb_dim = text_encoder_embedding_format(
    'openclip:hf-hub:apple/DFN5B-CLIP-ViT-H-14-378'
)

train_loader = build_streaming_latents_dataloader(
    datadir=[
        '/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/'
    ],
    image_size=512,
    batch_size=128 // dist.get_world_size(), # 2048
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    cap_drop_prob=0.0,
    drop_last=True,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

eval_loader = build_streaming_latents_dataloader(
    datadir='/root/.cache/micro_diffusion/data/mds_latents_sdxl1_dfnclipH14/train/',
    image_size=512,
    batch_size=1024 // dist.get_world_size(),
    cap_seq_size=cap_seq_size,
    cap_emb_dim=cap_emb_dim,
    drop_last=False,
    shuffle=True,
    prefetch_factor=None,
    num_workers=0,
    persistent_workers=False,
    pin_memory=True
)

In [ ]:
loggers = []

loggers.append(TensorboardLogger())
exp_name = 'MicroDiTXL_mask_0_res_512_finetune_final'
wandb_logger = WandBLogger(
    init_kwargs={
        'name': f'{exp_name}',
        'project': 'microdit_training',  # insert wandb project name
        'group': f'{exp_name}'
    }
)
# loggers.append(wandb_logger)

In [ ]:
algorithms = []
apply_low_precision_layernorm(
    model=model.dit,
    precision=Precision('amp_bf16'),
    optimizers=optimizer
)
algorithms.append(
    GradientClipping(clipping_type='norm', clipping_threshold=0.25)
)

In [ ]:
callbacks = []
callbacks.append(SpeedMonitor(window_size=3))
callbacks.append(LRMonitor())
callbacks.append(RuntimeEstimator())
callbacks.append(OptimizerMonitor())
callbacks.append(
    LogDiffusionImages(
        prompts=[
            "a photograph of an astronaut riding a horse",
            "An astronaut riding a pig, highly realistic dslr photo, cinematic shot",
            "Panda mad scientist mixing sparkling chemicals, artstation",
            "a close-up of a fire spitting dragon, cinematic shot.",
            "A small cactus with a happy face in the Sahara desert",
            "Pirate ship trapped in a cosmic maelstrom nebula, rendered in cosmic beach whirlpool engine, volumetric lighting, spectacular, ambient lights, light pollution, cinematic atmosphere, art nouveau style, illustration art artwork by SenseiJaye, intricate detail.",
            "A dog that has been meditating all the time",
            "A Pikachu with an angry expression and red eyes, with lightning around it, hyper realistic style",
            "A worker that looks like a mixture of cow and horse is working hard to type code",
            "A capybara made of lego sitting in a realistic, natural field",
            "A grand piano with a white bench.",
            "In a fantastical setting, a highly detailed furry humanoid skunk with piercing eyes confidently poses in a medium shot, wearing an animal hide jacket. The artist has masterfully rendered the character in digital art, capturing the intricate details of fur and clothing texture.",
            "A illustration from a graphic novel. A bustling city street under the shine of a full moon. The sidewalks bustling with pedestrians enjoying the nightlife. At the corner stall, a young woman with fiery red hair, dressed in a signature velvet cloak, is haggling with the grumpy old vendor. the grumpy vendor, a tall, sophisticated man is wearing a sharp suit, sports a noteworthy moustache is animatedly conversing on his steampunk telephone.",
            "A fierce garden gnome warrior, clad in armor crafted from leaves and bark, brandishes a tiny sword and shield. He stands valiantly on a rock amidst a blooming garden, surrounded by colorful flowers and towering plants. A determined expression is painted on his face, ready to defend his garden kingdom.",
            "A giant cobra snake made from corn",
            "A green sign that says 'Very Deep Learning' and is at the edge of the Grand Canyon. Puffy white clouds are in the sky"
        ],
        guidance_scale=5,
        sampling_steps=30,
        seed=42
    )
)
callbacks.append(NaNCatcher())

In [ ]:
scheduler = ConstantScheduler(alpha=1.0)

In [ ]:
trainer = Trainer(
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=eval_loader,
    optimizers=optimizer,
    schedulers=scheduler,
    loggers=loggers,
    algorithms=algorithms,
    callbacks=callbacks,
    device='gpu',
    # max_duration='55000ba',
    max_duration='1ba',
    eval_interval='500ba',
    save_interval='500ba',
    save_num_checkpoints_to_keep=1,
    device_train_microbatch_size=64,
    run_name='microdit_experiment',
    seed=18,
    save_folder='./trained_models/microdit_experiment/',
    load_path='./trained_models/MicroDiTXL_mask_0_res_256_finetune/latest-rank0.pt',
    load_weights_only=True,
    load_strict_model_weights=False,
    load_ignore_keys=["state/optimizers/AdamW/param_groups/initial_lr", "state/optimizers/AdamW/param_groups/lr", "state/schedulers/LambdaLR/base_lrs", "state/schedulers/LambdaLR/_last_lr"],
    save_overwrite=True,
    autoresume=False,
    parallelism_config=None,
    precision='amp_bf16',  # if cfg.model['dtype'] == 'bfloat16' else 'amp_fp16',  # fp16 by default
    python_log_level='debug',
    compile_config={} if compile else None,
    # fsdp_config=fsdp_config
)

In [ ]:
device = next(model.dit.parameters()).device
model.vae.to(device)
model.text_encoder.to(device)

In [ ]:
trainer.fit()